In [18]:
import sys 
import numpy as np
import cv2 

src = cv2.imread('polygon.bmp',cv2.IMREAD_GRAYSCALE)

_, src_bin = cv2.threshold(src,0,255,cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)

cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)

dst = cv2.cvtColor(src_bin, cv2.COLOR_GRAY2BGR)
# dst = np.zeros((h,w), dtype='uint8')

for i in range(1,cnt):
    (x,y,w,h,area) = stats[i]
    
    if area < 20:
        continue
    
    cv2.rectangle(dst, (x,y,w,h), (0,255,255))



cv2.imshow('src_bin',src_bin)
cv2.imshow('src',src)
cv2.imshow('dst',dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [23]:
import cv2 
import numpy as np
import random
src= cv2.imread('milkdrop.bmp',cv2.IMREAD_GRAYSCALE)

_, src_bin = cv2.threshold(src, 0,255, cv2.THRESH_OTSU)

contours, _ = cv2.findContours(src_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

h,w = src.shape[:2]
dst = np.zeros((h,w,3), np.uint8)

for i in range(len(contours)):
    area = cv2.contourArea(contours[i])
    if area > 600:
        c = (random.randint(0,255), random.randint(0,255), random.randint(0,255))
        cv2.drawContours(dst,contours,i,c,1,cv2.LINE_AA)
        m = cv2.moments(points)
        x = m['m10']/m['m00']
        y = m['m01']/m['m00']
        cv2.circle(dst,(int(x),int(y)),3,c,-1)
        
        
        
cv2.imshow('src',src)
cv2.imshow('src_bin',src_bin)
cv2.imshow('dst',dst)
cv2.waitKey()
cv2.destroyAllWindows()

NameError: name 'points' is not defined

In [26]:
import random
import numpy as np
import cv2

src = cv2.imread('milkdrop.bmp', cv2.IMREAD_GRAYSCALE)
_, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_OTSU)
contours, _ = cv2.findContours(src_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

h, w = src.shape[:2]
dst = np.zeros((h, w, 3), np.uint8)

for i in range(len(contours)):
    points = contours[i]
    area = cv2.contourArea(points)
    if area > 600:
        #외곽선 그리기
        c = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        cv2.drawContours(dst, contours, i, c, 1, cv2.LINE_AA)
        
        #외곽선으로 모멘트 계산
        m = cv2.moments(points)
        
        #외곽선의 중심점 좌표
        x = m['m10']/m['m00']
        y = m['m01']/m['m00']
        cv2.circle(dst, (int(x),int(y)), 3, c, -1)
        
        #외곽선 둘레 * 0.01
        p1 = 0.01 * cv2.arcLength(points, True)
        #외곽선 근사화(점의 수를 줄임)
        ap = cv2.approxPolyDP(points, p1, True)
        c = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        #계산된 근사치 좌표로 외곽선 그림
        cv2.drawContours(dst, [ap], 0, c, 1, cv2.LINE_AA)
        
cv2.imshow('src', src)
cv2.imshow('src_bin', src_bin)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

폴리곤 이미지 파일을 읽어서 \
컨벡스 헐 처리 후 좌표의 개수로 \
삼각형, 사각형, 다각형을 구분하는 코드 작성

In [30]:
import math
import cv2 

def setLabel(img,pts, label):
    (x,y,w,h) = cv2.boundingRect(pts)
    pt1 = (x,y)
    pt2 = (x+w, y+h)
    cv2.rectangle(img,pt1,pt2,(0,0,255),1)
    cv2.putText(img, label,pt1, cv2.FONT_HERSHEY_PLAIN,1, (0,0,255))
    
def main():
    img = cv2.imread('polygon.bmp',cv2.IMREAD_COLOR)
    
    if img is None:
        print('Image load failed')
        return 
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    _, img_bin = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    for pts in contours:
        if cv2.contourArea(pts) < 400: # 너무 작으면 무시
            continue
            
        approx = cv2.approxPolyDP(pts,cv2.arcLength(pts,True)*0.02,True)
        
        vtc = len(approx)
        
        if vtc == 3:
            setLabel(img,pts,'TRI')
        elif vtc == 4:
            setLabel(img,pts,'RECT')
        else:
            length = cv2.arcLength(pts,True)
            area = cv2.contourArea(pts)
            ratio = 4. * math.pi * area / (length*length)
            
            if ratio > 0.85 :
                setLabel(img,pts,'CIR')
        
    cv2.imshow('img', img)
    cv2.waitKey()
    cv2.destroyAllWindows()


main()
                

In [34]:
import math
import cv2 

src = cv2.imread('polygon.bmp',cv2.IMREAD_GRAYSCALE)
_, src_bin = cv2.threshold(src,0,255,cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
contours, _ = cv2.findContours(src_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

h,w = src.shape[:2]
dst = np.zeros((h,w,3),np.uint8)

for cont in contours:
    area = cv2.contourArea(cont)
    if area > 600:
        cv2.drawContours(dst,[cont], 0 , (255,0,0),1,cv2.LINE_AA)
        p1 = 0.01 * cv2.arcLength(cont,True)
        ap = cv2.approxPolyDP(cont,p1,True)
        hull = cv2.convexHull(ap)
        p_cnt = len(hull)
        if p_cnt ==3:
            label='삼각형'
        elif p_cnt == 4:
            label = '사각형'
        elif 4< p_cnt < 10:
            label = '다각형'
        elif p_cnt >=10:
            label='원'
            
        cv2.drawContours(dst, [hull],0,(0,0,255),1,cv2.LINE_AA)
        
cv2.imshow('dst',dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [36]:
import numpy as np
import cv2

src = cv2.imread('milkdrop.bmp', cv2.IMREAD_GRAYSCALE)
_, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_OTSU)
contours, _ = cv2.findContours(src_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

h, w = src.shape[:2]
dst = np.zeros((h, w, 3), np.uint8)

for cont in contours:
    area = cv2.contourArea(cont)
    if area > 600:
        cv2.drawContours(dst, [cont], 0, (255, 0, 0), 1, cv2.LINE_AA)
        hull = cv2.convexHull(cont)
        cv2.drawContours(dst, [hull], 0, (0, 0, 255), 1, cv2.LINE_AA)
        
cv2.imshow('src', src)
cv2.imshow('src_bin', src_bin)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()



<도형 구분>
'''
폴리곤 이미지 파일을 읽어서
컨벡스 헐 처리 후 좌표의 개수로
삼각형, 사각형, 다각형을 구분하는 코드 작성
'''

In [39]:

import numpy as np
import cv2

src = cv2.imread('polygon.bmp', cv2.IMREAD_GRAYSCALE)
_, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
contours, _ = cv2.findContours(src_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

h, w = src.shape[:2]
dst = np.zeros((h, w, 3), np.uint8)

for cont in contours:
    area = cv2.contourArea(cont)
    if area > 600:
        cv2.drawContours(dst, [cont], 0, (255, 0, 0), 1, cv2.LINE_AA)
        p1 = 0.01 * cv2.arcLength(cont, True)
        ap = cv2.approxPolyDP(cont, p1, True)
        hull = cv2.convexHull(ap)
        p_cnt = len(hull)
        if p_cnt==3:
            label = 'tri'
        elif p_cnt == 4:
            label = 'rect'
        elif 4 < p_cnt < 10:
            label = 'poly'
        elif p_cnt >=10 :
            label = 'circle'
        
        cv2.drawContours(dst, [hull], 0, (0, 0, 255), 1, cv2.LINE_AA)
        x = cont[cont[:,:,0].argmin()][0][0]
        y = cont[cont[:,:,1].argmin()][0][1]
        cv2.putText(dst, label, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (200,200,200))
        print(x, y)
        
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

65 285
404 275
247 271
440 205
139 166
427 71
263 58
70 34


In [40]:

import numpy as np
import cv2

src = cv2.imread('polygon.bmp', cv2.IMREAD_GRAYSCALE)
_, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
contours, _ = cv2.findContours(src_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

h, w = src.shape[:2]
dst = np.zeros((h, w, 3), np.uint8)

for cont in contours:
    area = cv2.contourArea(cont)
    if area > 600:
        cv2.drawContours(dst,[cont],0,(255,0,0),1,cv2.LINE_AA)
        x_min = cont[cont[:,:,0].argmin()][0]
        y_min = cont[cont[:,:,1].argmin()][0]
        x_max = cont[cont[:,:,0].argmax()][0]
        y_max = cont[cont[:,:,1].argmax()][0]
        
        cv2.circle(dst,(x_min[0],x_min[1]),3,(0,0,255),-1)
        cv2.circle(dst,(y_min[0],y_min[1]),3,(0,0,255),-1)
        cv2.circle(dst,(x_max[0],x_max[1]),3,(0,0,255),-1)
        cv2.circle(dst,(y_max[0],y_max[1]),3,(0,0,255),-1)
        
cv2.imshow('dst',dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [44]:
cont[0][0][0]

186

In [ ]:
import sys 
import numpy as np
import cv2 

src = cv2.imread('polygon.bmp',)